In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
%%capture   
%pwd


In [ ]:
%%capture
cd ..

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import psycopg2 as ps2
import os
import pandas as pd
import numpy as np
import csv
from src.funcs.utils import *

from dotenv import load_dotenv
load_dotenv()

In [ ]:
# pull in db credentials from .env file
db_name=os.getenv('DB_NAME')
db_user=os.getenv('DB_USER')
db_host=os.getenv('DB_HOST_IP')
db_pwd=os.getenv('DB_PWD')

In [ ]:
# conn.close()
# conn.rollback()

In [ ]:
# define db session
conn = ps2.connect(
    host = db_host,
    database = db_name,
    user = db_user,
    password = db_pwd)

## Relationships

### Query for data

### Directional

In [ ]:
sql="SELECT \
control.id, inkey[1], controltype,  string_agg(distinct(effect), ', '), string_agg(distinct(mechanism), ', '), \
num_refs, outkey[1] , reference.id, string_agg(distinct(biomarkertype), ', ') , string_agg(distinct(celllinename), ', '), \
string_agg(distinct(celltype), ', '), string_agg(distinct(changetype), ', '), string_agg(distinct(organ), ', '), \
string_agg(distinct(organism), ', '), string_agg(distinct(quantitativetype), ', '), string_agg(distinct(tissue), ', '),  \
string_agg(distinct(nct_id), ', '),  string_agg(distinct(phase), ', ') \
FROM resnet18.control, resnet18.reference \
WHERE control.id = reference.id and inkey[1] is not null and outkey[1] is not null \
GROUP BY control.id, inkey[1], controltype, num_refs, outkey[1], reference.id"

In [ ]:
%%time
# run query and save to file
with conn.cursor() as cur:
    with open('./data/processed/directional_rels_raw.txt', 'w', encoding="utf-8") as f:
        cur.execute(sql)
        csv_writer = csv.writer(f, delimiter='|')
        for record in cur.fetchall():
            csv_writer.writerow(record)  

### Bi-derectional

In [ ]:
sql="SELECT \
control.id, inkey[1], inoutkey, controltype, relationship, string_agg(distinct(effect), ', '), string_agg(distinct(mechanism), ', '), num_refs, outkey[1], \
reference.id, string_agg(distinct(biomarkertype), ', '), string_agg(distinct(celllinename), ', '), string_agg(distinct(celltype), ', '), string_agg(distinct(changetype), ', '), \
string_agg(distinct(organ), ', '), string_agg(distinct(organism), ', '), string_agg(distinct(quantitativetype), ', '), string_agg(distinct(tissue), ', ') \
FROM resnet18.control, resnet18.reference \
WHERE control.id = reference.id and inkey[1] is null and outkey[1] is null \
GROUP BY control.id, controltype, reference.id"

In [ ]:
%%time
# run query and save to file
with conn.cursor() as cur:
    with open('./data/processed/bidirectional_rels_raw.txt', 'w', encoding="utf-8") as f:
        cur.execute(sql)
        csv_writer=csv.writer(f, delimiter= '|')
        for record in cur.fetchall():
            line='|'.join(map(str, record))+'\n'
            f.write(line)

### Attributes

In [ ]:
sql="SELECT  \
id, inkey[1], attributes, relationship, outkey[1] from resnet18.control \
WHERE (control.id = control.attributes and inkey[1] is not null and outkey[1] is not null)";

In [ ]:
%%time
with conn.cursor() as cur:
    with open('./data/processed/attribute_rels_raw.txt', 'w', encoding="utf-8") as f:
        cur.execute(sql)
        csv_writer=csv.writer(f, delimiter= '|')
        for record in cur.fetchall():
            line='|'.join(map(str, record))+'\n'
            f.write(line)

## Process Data

### Bi-directional

In [ ]:
%%time

df_birect=pd.read_csv('./data/processed/bidirectional_rels_raw.txt', sep='|', header=None, encoding='utf-8')
df_birect.info(); df_birect.head()

In [ ]:
df_birect.columns=['msrc_id', ':START_ID', 'inOutkey', 'type:TYPE', 'relationship', 'effect', 'mechanism', 'ref_count:int', ':END_ID' , 'id2', 'biomarkertype', 'celllinename', 'celltype',
      'changetype', 'organ', 'organism', 'quantitativetype', 'tissue']

df_birect.head()

In [ ]:
# create separate columns for inOutkeys
first_ids, second_ids = inOutkeys_to_lists(df_birect)

len(first_ids); len(second_ids)
first_ids[:5]; second_ids[:5]

In [ ]:
df_birect_1=df_birect.copy()
df_birect_1.drop(columns=['inOutkey', 'id2', 'relationship'], inplace=True)

In [ ]:
df_birect_1[':START_ID']=first_ids
df_birect_1[':END_ID']=second_ids
df_birect_1.info(); df_birect_1.head()

In [ ]:
df_birect_1[':START_ID']=df_birect_1[':START_ID'].astype('int64')
df_birect_1[':END_ID']=df_birect_1[':END_ID'].astype('int64')
df_birect_1['ref_count:int']=df_birect_1['ref_count:int'].astype('int16')

df_birect_1.info()

In [ ]:
# convert to category to save memory
df_birect_1=convert_object_to_category(df_birect_1)
df_birect_1.info()

In [ ]:
df_birect_1.to_pickle('./data/processed/bidirectional_rels_procd.pkl')

### Directional

In [ ]:
%%time

df_direct=pd.read_csv('./data/processed/directional_rels_raw.txt', sep='|', encoding='utf-8')

df_direct.info(); df_direct.head()

In [ ]:
df_direct.columns=['msrc_id', ':START_ID', 'type:TYPE', 'effect', 'mechanism', 'ref_count:int', ':END_ID' , 'id2', \
            'biomarkertype', 'celllinename', 'celltype', 'changetype', 'organ', 'organism', 'quantitativetype', \
            'tissue', 'nct_id', 'phase']

In [ ]:
df_direct.info(); df_direct.head()

In [ ]:
df_direct=df_direct.drop(columns=['id2'])

df_direct.head()

In [ ]:
df_direct['phase']=df_direct['phase'].fillna('None')

In [ ]:
# convert to category dtype to save memory
df_direct=convert_object_to_category(df_direct)
    
df_direct.info()

In [ ]:
df_direct['ref_count:int']=df_direct['ref_count:int'].astype('int16')

In [ ]:
df_direct.isnull().sum()

In [ ]:
df_direct['type:TYPE'].value_counts()

In [ ]:
df_direct.to_pickle('./data/processed/directional_rels_procd.pkl')

### Attributes

In [ ]:
%%time

df_att=pd.read_csv('./data/processed/attribute_rels_raw.txt', sep='|', header=None, encoding='utf-8')

df_att.info(); df_att.head()

In [ ]:
df_att.columns=['msrc_id', ':START_ID', 'id2', 'type:TYPE', ':END_ID']

df_att.info()

In [ ]:
df_att['type:TYPE'].value_counts()

In [ ]:
df_att=df_att.dropna(how='any')

df_att.reset_index(drop=True, inplace=True)
df_att. info(); df_att.head()

In [ ]:
df_att=df_att.drop(columns=['id2'])

df_att.info(); df_att.head()

In [ ]:
# remove any unwanted white space
for col in list(df_att.columns):
    df_att[col]=df_att[col].apply(lambda x: str(x).strip())

In [ ]:
df_att[':START_ID'] = pd.to_numeric(df_att[':START_ID'], errors='coerce').astype('int64')
df_att[':END_ID'] = pd.to_numeric(df_att[':END_ID'], errors='coerce').astype('int64')
df_att['type:TYPE']=df_att['type:TYPE'].astype('category')


In [ ]:
df_att.head()

In [ ]:
df_att['type:TYPE'].value_counts()

In [ ]:
df_att.to_pickle('./data/processed/attribute_rels_procd.pkl')

### Concatenate Relationships

In [ ]:
df_directional=pd.read_pickle('./data/processed/directional_rels_procd.pkl')
df_directional.head(); df_directional.info()

In [ ]:
df_bidirect=pd.read_pickle('./data/processed/bidirectional_rels_procd.pkl')
df_bidirect.head(); df_bidirect.info()

In [ ]:
df_att=pd.read_pickle('./data/processed/attribute_rels_procd.pkl')
df_att.head(); df_att.info()

In [ ]:
%%time

df_concat=pd.concat([df_directional, df_bidirect, df_att])

In [ ]:
df_concat.info(); df_concat.head()

In [ ]:
for col in df_concat.select_dtypes(include=['category']).columns:
    df_concat[col] = df_concat[col].astype('object')

In [ ]:
df_concat.info()

In [ ]:
df_concat.isnull().sum()

In [ ]:
df_concat['ref_count:int'].fillna(0, inplace=True)

In [ ]:
df_concat=df_concat.fillna('None')

In [ ]:
df_concat=df_concat.replace('nan', 'None')

In [ ]:
df_concat.head(); df_concat.info()

In [ ]:
# set ref count dtype
df_concat['ref_count:int']=df_concat['ref_count:int'].astype('int16')
df_concat['msrc_id']=df_concat['msrc_id'].astype('int64')

In [ ]:
# change object to category to save memory
df_concat=convert_object_to_category(df_concat)

df_concat.info()

In [ ]:
df_concat.reset_index(drop=True, inplace=True)
df_concat.info()

In [ ]:
df_concat.drop_duplicates(inplace=True)
df_concat.reset_index(drop=True, inplace=True)

df_concat.info()

In [ ]:
df_concat['type:TYPE'].value_counts()
df_concat['organ'].value_counts()

In [ ]:
df_concat=df_concat.replace('None', '_')
df_concat.info(); df_concat.head()

In [ ]:
df_concat['type:TYPE']=df_concat['type:TYPE'].apply(lambda x: x.upper())
df_concat.info(); df_concat.head()

In [ ]:
df_concat[df_concat['ref_count:int']==0]['type:TYPE'].value_counts()

In [ ]:
df_concat['type:TYPE'].value_counts()

In [ ]:
cols=list(df_directional.columns)
cols

In [ ]:
df_concat=df_concat[cols]
df_concat.head(5)

In [ ]:
%%time
df_concat.to_csv('./data/processed/relations.txt', sep='|', index=False, header=True)

In [ ]:
# list of column names for header file
cols=list(df_concat.columns)
cols

In [ ]:
# create headers file
df_rel_headers=pd.DataFrame(columns=cols)
df_rel_headers
df_rel_headers.to_csv('./data/processed/relations_header.txt', sep= '|', index=False)

## Nodes

In [ ]:
# define query
sql='select id, name, nodetype from resnet18.node where id is not null and name is not null and nodetype is not null'


In [ ]:
%%time

# save query results to file
with conn.cursor() as cur:
    with open('./data/raw/nodes_raw.txt', 'w', encoding="utf-8") as f:
        cur.execute(sql)
        csv_writer=csv.writer(f, delimiter='|')
        for record in cur.fetchall():
            line="|".join(map(str, record))+'\n'
            
            f.write(line)

### Create Node files

In [ ]:
df_node=pd.read_csv('./data/raw/nodes_raw.txt', delimiter='|', header=None, encoding='utf-8')

df_node.info(); df_node.head()

In [ ]:
# create column headers
df_node.columns=[':ID', 'name', ':LABEL']
df_node.head()

In [ ]:
# df cleanup
df_node[':LABEL']=df_node[':LABEL'].apply(lambda x: x.upper())
df_node[':LABEL']=df_node[':LABEL'].apply(lambda x: x.strip())
df_node['name']=df_node['name'].apply(lambda x: x.strip())
df_node[':ID']=df_node[':ID'].apply(lambda x: str(x).strip())  #convert to string and strip()
df_node[':ID']=df_node[':ID'].astype('int64')   # convert back to int
df_node.info(); df_node.head()

In [ ]:
# check for name strings that are too long
df_node['name_len']=df_node['name'].apply(lambda x: len(x))

In [ ]:
df_node.sort_values(by='name_len', ascending=False)

In [ ]:
# Does not seem to be an issue when preprocessing in a Linux environment
# Issue is leading quote with no ending quote in sql db for id -7235442027224814239 above
# the following is a temp fix unitil resolved in sql db
# process long name field

x=df_node[df_node[':ID']==-7235442027224814239]['name']
long_index=df_node[df_node[':ID']==-7235442027224814239].index[0] #capture index to delete later
long_index
x=x.to_frame().reset_index()
x.info(); x.head()
y=x.loc[0,'name']
len(y)

In [ ]:
# parse bad field
lst=y.split('\r\n')
lst=y.split('\n') # use this if above fails

lst[:5]
lst[0]='-7235442027224814239|' + lst[0]
lst[:5]

In [ ]:
# create list of lists from long name field
new_list=[]
for i in range(len(lst)):
    z=lst[i].split('|')
    new_list.append(z)
    

In [ ]:
len(new_list)
new_list[:5]

In [ ]:
# create df from list of lists
df_temp=pd.DataFrame(new_list, columns=[':ID', 'name', ':LABEL'])
df_temp.info(); df_temp.head()

In [ ]:
df_temp.dropna(inplace=True)
df_temp.info()

In [ ]:
df_temp[':LABEL']=df_temp[':LABEL'].apply(lambda x: x.upper())
df_temp.head()

In [ ]:
# combine dfs
df_node.drop(int(f'{long_index}'), inplace = True) #drop row with long name field
df_node.drop(columns=['name_len'], inplace=True)

df_node_new=pd.concat([df_node, df_temp])
df_node_new.reset_index(inplace=True, drop=True)
df_node_new.info(); df_node_new.head()

In [ ]:
# check for presence of ';;' & ';'
df_node_new[df_node_new['name'].str.contains(";")]
df_node_new[df_node_new['name'].str.contains(";;")]

In [ ]:
# convert single and double semi-colons to colons
df_node_new['name']=df_node_new['name'].str.replace(';;', ':')
df_node_new['name']=df_node_new['name'].str.replace(';', ':')


In [ ]:
df_node_new.to_csv('./data/processed/nodes.txt', sep='|', index=False, header=True)

In [ ]:
# create list of column names for header file
cols=list(df_node.columns)
cols

In [ ]:
# create and save header file
node_headers=pd.DataFrame(columns=cols)
node_headers
node_headers.to_csv('./data/processed/nodes_header.txt', sep='|', index=False)

In [ ]:
conn.close()

**To load data into Neo4j**, click on 'Add' in the project pane. Name the new project, click on the '...' by the 'Open' button for the project, and choose 'Terminal'. Take note of the dbms number at the cursor. Copy the nodes.txt, nodes_header.txt, relations.txt, and realations_header.txt files pepared above. Drop them in the import file found here: C:\Users\\[user_name]\\.Neo4jDesktop\relate-data\dbmss\dbms-##(number at cursor)\import

At the Neo4j terminal, change to the bin directory (type 'cd bin') and then paste the following at the cursor:
neo4j-admin import --delimiter="|" --nodes=import/nodes_header.txt, import/nodes.txt --relationships=import/relations_header.txt, import/relations.txt --skip-bad-relationships=true

Go back to the project pane, press 'Start'. The active database will now be shown in a pane at the top. Press open to use in the Neo4j browser.